In [ ]:
#| hide
from nbdev.showdoc import *

# core

> Notion Automation for multiple page creation

In [ ]:
#| default_exp core

In [ ]:
#| export 
class Database:
    """Base class for interacting with a Notion database."""
    def __init__(self, 
            db_id, # Notion database ID
            notion # Notion client instance 
        ):
        self.db_id, self.notion = db_id, notion

        # Fetch and store the data source ID
        db_info = self.notion.databases.retrieve(self.db_id)
        self.data_source_id = db_info['data_sources'][0]['id']
    
    def get_schema(self):
        """Retrieve the database schema.
        
        Returns:
            dict: Mapping of property names to their types
        """
        db = self.notion.databases.retrieve(self.db_id)
        return {name: props['type'] for name, props in db['properties'].items()}
    
    def check_db_exists(self):
        "Check if database exists and is accessible"
        try:
            db = self.notion.databases.retrieve(database_id=self.db_id)
            print(f"✓ Database exists: {db['title'][0]['plain_text']}")
            return True
        except Exception as e:
            print(f"✗ Database not found: {e}")
            return False

In [ ]:
#| export
#| Export
class TriggerDB(Database):
    """
    Database that triggers logging when updated.
    """

    def __init__(self, 
                db_id, # Notion database ID 
                notion, # Notion client instance 
                status_prop, # Name of status property
                relation_prop, # Name of relation property
                qty_prop # Name of the quantity property
        ):
        """Initialize a TriggerDB instance.
        """
        super().__init__(db_id, notion)
        self.status_prop, self.relation_prop, self.qty_prop = status_prop, relation_prop, qty_prop
    
    #| exec_doc
    def get_page_data(self, 
                    page_id # Notion page ID to retrieve (the row that trigger change)
        ):
        """Extract status, related item ID, and quantity from a page.
        """
        page = self.notion.pages.retrieve(page_id)
        status = page['properties'][self.status_prop]['select']['name'] if page['properties'][self.status_prop]['select'] else None
        relation_id = page['properties'][self.relation_prop]['relation'][0]['id'] if page['properties'][self.relation_prop]['relation'] else None
        qty = page['properties'][self.qty_prop]['number']
        return status, relation_id, qty

In [ ]:
#| export
class JunctionDB(Database):
    """Database that connects trigger items to log items with amounts.
    
    Acts as a many-to-many relationship table, storing which items are affected
    by a trigger and their respective amounts or multipliers.
    """
    def __init__(self, 
                db_id, # Notion database ID
                notion, # Notion client instance 
                relation_prop, # Name of relation property to trigger items 
                item_prop, # Name of relation property to log items 
                amount_prop # Name of amount propert 
        ):
      
        super().__init__(db_id, notion)
        self.relation_prop, self.item_prop, self.amount_prop = relation_prop, item_prop, amount_prop
    
    def get_items(self, 
                relation_id # Trigger item's relation ID 
        ):
        """Get all items and their amounts for a given relation.

        Returns:
            dict: Mapping of item IDs to their amounts, empty dict if no relation_id
        """
        if not relation_id: return {}
        res = self.notion.data_sources.query(data_source_id=self.data_source_id, filter={"property": self.relation_prop, "relation": {"contains": relation_id}})
        return {r['properties'][self.item_prop]['relation'][0]['id']: r['properties'][self.amount_prop]['number'] for r in res['results']}

#| export
class LogDB(Database):
    """Database where transaction logs are written.
    
    Records all inventory or quantity changes with item references, amounts,
    triggering events, and reasons for the change.
    """

    def __init__(self, 
                db_id, # Notion database ID 
                notion, # Notion client instance 
                item_prop, # Name of item relation property 
                amount_prop, # Name of the amount property 
                trigger_prop, # Name of the trigger relation property 
                reason_prop # Name of the reason select property
            ):
        """Initialize a LogDB instance.
        """
        super().__init__(db_id, notion)
        self.item_prop, self.amount_prop, self.trigger_prop, self.reason_prop = item_prop, amount_prop, trigger_prop, reason_prop
    
    def create_entry(self, 
                    item_id, # ID of item being logged     
                    amount, # Amount to log 
                    trigger_id, # ID of triggering page  
                    reason # Reason for log entry 
        ):
        """Create a new log entry.
        """
        self.notion.pages.create(parent={"database_id": self.db_id}, properties={self.item_prop: {"relation": [{"id": item_id}]}, self.amount_prop: {"number": amount}, self.trigger_prop: {"relation": [{"id": trigger_id}]}, self.reason_prop: {"select": {"name": reason}}})



In [ ]:
#| export
class AutoLogger:
    """Orchestrates automatic logging from trigger to log via junction.
    
    Monitors trigger database changes and automatically creates corresponding
    log entries by looking up related items through the junction database.
    """
    def __init__(self, 
                trigger_db, # TriggerDB instance
                junction_db, # JunctionDB instance
                log_db, # LogDB instance
                trigger_status, # Status value that triggers logging
                multiplier=-1 # Amount multiplier (typically -1 for deduction)
        ):
        self.trigger_db, self.junction_db, self.log_db = trigger_db, junction_db, log_db
        self.trigger_status, self.multiplier = trigger_status, multiplier
    
    def process(self, 
        page_id,
        reason:str # Reason for log entry
        ):
        """Process a trigger page and create log entries."""
        status, relation_id, qty = self.trigger_db.get_page_data(page_id)
        if status != self.trigger_status: return f"Status is {status}, not {self.trigger_status}"
        items = self.junction_db.get_items(relation_id)
        if not items: return "No items found in junction"
        for item_id, amt in items.items(): self.log_db.create_entry(item_id, self.multiplier * qty * amt, page_id, reason)
        return f"Logged {len(items)} items"
    
    def cancel(self, page_id):
        """Reverse all logs for a trigger (restore inventory)."""
        _, relation_id, qty = self.trigger_db.get_page_data(page_id)
        items = self.junction_db.get_items(relation_id)
        if not items: return "No items to reverse"
        for item_id, amt in items.items(): self.log_db.create_entry(item_id, -self.multiplier * qty * amt, page_id, 'cancelled')
        return f"Reversed {len(items)} items"
    
    def adjust_batch(self, 
                    page_id, # Trigger page ID being adjusted
                    old_qty, # Previous quantity value
                    new_qty # New quantity value
        ):
        """Log the difference when batch quantity changes."""
        status, relation_id, _ = self.trigger_db.get_page_data(page_id)
        if status != self.trigger_status: return f"Status is {status}, not active"
        items = self.junction_db.get_items(relation_id)
        if not items: return "No items found"
        delta = new_qty - old_qty
        for item_id, amt in items.items(): self.log_db.create_entry(item_id, self.multiplier * delta * amt, page_id, 'batch_adjusted')
        return f"Adjusted {len(items)} items by {delta} batches"
    
    def handle_update(self, 
                    page_id, # Trigger page ID that was updated
                    old_status=None, # Previous status value
                    old_qty=None # Previous quantity value
        ):
        """Handle any update to trigger page - detects what changed and acts accordingly."""
        status, _, qty = self.trigger_db.get_page_data(page_id)
        if old_status and old_status == self.trigger_status and status != self.trigger_status: return self.cancel(page_id)
        if status == self.trigger_status:
            if old_status and old_status != self.trigger_status: return self.process(page_id)
            if old_qty and old_qty != qty: return self.adjust_batch(page_id, old_qty, qty)
        return "No action needed"


In [ ]:
#|hide 
import nbdev; nbdev.nbdev_export()